---
title: "How to build a motion tracking pipeline: 2D pose estimation via OpenPose"
authors: Šárka Kadavá
---


In this script, we will look into another way how to capture motion. 

There are some significant differences when compared to MediaPipe:

- It requires GPU to run
- It does **not** provide 3D pose estimation 
- It can track multiple people at once

## Useful links

Demo of this pipeline has been published on [EnvisionBOX](https://www.envisionbox.org/embedded_openpose_to_pose2sim_tracking.html)

See [OpenPose documentation](https://github.com/CMU-Perceptual-Computing-Lab/openpose) for more information.

Once again, we first prepare our environment

In [ ]:
import os
import subprocess
import glob
import tempfile
from IPython.display import Video
import random

curfolder = os.getcwd()

# Openpose demo.exe location
openposefol = curfolder+'/openpose/'
openpose_demo_loc = openposefol + '/bin/OpenPoseDemo.exe'

# This is the model to employ - with 135 keypoints
model_to_employ = 'BODY_135'

# List folders in a main folder
folderstotrack = glob.glob(curfolder +'/projectdata/*')

# Get all folders per participant, per session
pcnfolders = []

for i in folderstotrack:
    pcn1folders = glob.glob(i + '/P0/*')
    pcn2folders = glob.glob(i + '/P1/*')
    pcnfolders_in_session = pcn1folders + pcn2folders

    # Append to the list
    pcnfolders = pcnfolders + pcnfolders_in_session

# There might be some other things we don't want now
pcnfolders = [x for x in pcnfolders if 'Config' not in x]

print(pcnfolders[0:10])

['c:\\Users\\Sarka Kadava\\Documents\\Github\\MotionTrackingPipeline_Workshop_ZAS2025\\01_MotionCapture\\OpenPose_pose2sim/projectdata\\Session_0_1/P0\\0_1_20_p0', 'c:\\Users\\Sarka Kadava\\Documents\\Github\\MotionTrackingPipeline_Workshop_ZAS2025\\01_MotionCapture\\OpenPose_pose2sim/projectdata\\Session_0_1/P0\\0_1_21_p0', 'c:\\Users\\Sarka Kadava\\Documents\\Github\\MotionTrackingPipeline_Workshop_ZAS2025\\01_MotionCapture\\OpenPose_pose2sim/projectdata\\Session_0_1/P0\\0_1_22_p0', 'c:\\Users\\Sarka Kadava\\Documents\\Github\\MotionTrackingPipeline_Workshop_ZAS2025\\01_MotionCapture\\OpenPose_pose2sim/projectdata\\Session_0_1/P0\\0_1_23_p0', 'c:\\Users\\Sarka Kadava\\Documents\\Github\\MotionTrackingPipeline_Workshop_ZAS2025\\01_MotionCapture\\OpenPose_pose2sim/projectdata\\Session_0_1/P0\\0_1_24_p0', 'c:\\Users\\Sarka Kadava\\Documents\\Github\\MotionTrackingPipeline_Workshop_ZAS2025\\01_MotionCapture\\OpenPose_pose2sim/projectdata\\Session_0_1/P0\\0_1_25_p0', 'c:\\Users\\Sarka Kad

This time we will not use jupyter notebook as such, but we will 'send' a command into the terminal. 

In [4]:
import subprocess

def runcommand(command):
    try:
        result = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Command execution failed with error code {e.returncode}")
        print("Standard Error Output:", e.stderr)
    except FileNotFoundError:
        print("The OpenPoseDemo.exe executable was not found.")


In [6]:

for i in pcnfolders:
    os.chdir(openposefol)
    print('working on ' + i)

    # Identify all avi files in folder
    direc = glob.glob(i + '/raw-2d/' +'*.avi')

    # 3 cameras
    video0 = direc[0]
    video1 = direc[1]
    video2 = direc[2]

    videolist = [video0, video1, video2]
    
    # Make a new directory if it doesn't exist
    if not os.path.exists(i+'/pose/'):
        os.makedirs(i+'/pose/')
    if not os.path.exists(i+'/pose/pose_cam1_json/'):
        os.makedirs(i+'/pose/pose_cam1_json/')
    if not os.path.exists(i+'/pose/pose_cam2_json/'):
        os.makedirs(i+'/pose/pose_cam2_json/')
    if not os.path.exists(i+'/pose/pose_cam3_json/'):
        os.makedirs(i+'/pose/pose_cam3_json/')
    

    # Also make directory for openpose videos (pose-2d-trackingvideos)
    if not os.path.exists(i+'/pose-2d-trackingvideos/'):
        os.makedirs(i+'/pose-2d-trackingvideos/')

    # Initialize the pose2 folder
    outputfol1 = i+'/pose/pose_cam1_json/'
    outputfol2 = i+'/pose/pose_cam2_json/'
    outputfol3 = i+'/pose/pose_cam3_json/'

    outputfollist = [outputfol1, outputfol2, outputfol3]

    for it, j in enumerate(outputfollist):
        # Prepare the command
        openposelocation = f'"{openpose_demo_loc}" '
        model = f'--model_pose {model_to_employ} '
        video = f'--video "{videolist[it]}" '
        todo = '--write_json '
        outputfol = f'"{j}" '
        videoadd = '--write_video '
        videopath = f'"{i}/pose-2d-trackingvideos/video{it}.avi" '

        # Construct the full command
        command = f'{openposelocation}{model}{video}{todo}{outputfol}{videoadd}{videopath}'
        
        print('were going to send this to command prompt: ' + command)
        runcommand(command)

    

working on c:\Users\Sarka Kadava\Documents\Github\MotionTrackingPipeline_Workshop_ZAS2025\01_MotionCapture\OpenPose_pose2sim/projectdata\Session_0_1/P0\0_1_20_p0
were going to send this to command prompt: "c:\Users\Sarka Kadava\Documents\Github\MotionTrackingPipeline_Workshop_ZAS2025\01_MotionCapture\OpenPose_pose2sim/openpose//bin/OpenPoseDemo.exe" --model_pose BODY_135 --video "c:\Users\Sarka Kadava\Documents\Github\MotionTrackingPipeline_Workshop_ZAS2025\01_MotionCapture\OpenPose_pose2sim/projectdata\Session_0_1/P0\0_1_20_p0/raw-2d\0_1_trial_20_p0_verdrietig_combinatie_video_raw_cam1.avi" --write_json "c:\Users\Sarka Kadava\Documents\Github\MotionTrackingPipeline_Workshop_ZAS2025\01_MotionCapture\OpenPose_pose2sim/projectdata\Session_0_1/P0\0_1_20_p0/pose/pose_cam1_json/" --write_video "c:\Users\Sarka Kadava\Documents\Github\MotionTrackingPipeline_Workshop_ZAS2025\01_MotionCapture\OpenPose_pose2sim/projectdata\Session_0_1/P0\0_1_20_p0/pose-2d-trackingvideos/video0.avi" 
Command exec

KeyboardInterrupt: 

Similarly to MediaPipe, we have videos with fitted body skeleton. Here is an example of the output:

In [7]:
# Create a temporary folder
temp_dir = tempfile.mkdtemp()

openposevideos = glob.glob(curfolder + '\\projectdata\\*\\*\\*\\pose-2d-trackingvideos\\*.avi', recursive=True)

# Define input and output paths
sample = random.choice(openposevideos)
mp4_path = os.path.join(temp_dir, "converted_audiovideo.mp4")  # Save MP4 in temp folder

# Convert AVI to MP4 with video and audio
if not os.path.exists(mp4_path):
    subprocess.run([
        "ffmpeg", "-i", sample, 
        "-vcodec", "libx264", "-acodec", "aac", "-b:a", "192k",  # Ensure audio is included
        "-crf", "23", "-preset", "fast", mp4_path
    ])

# Display the video inside Jupyter Notebook
Video(mp4_path, embed=True, width=400)

The results are saved in  /pose folder in json format. But we are not done yet, now we have 2D coordinates from 3 videos. Next step is to use pose2sim to convert these 2D coordinates into 3D coordinates.